In [ ]:
import transformers
import torch
from transformers import AutoTokenizer, AutoModel

# Load your tokenizer and model
tokenizer = AutoTokenizer.from_pretrained('your-model-name')
model = AutoModel.from_pretrained('your-model-name')

assert len(tokenizer) == model.config.vocab_size, "Tokenizer vocab size does not match model vocab size"

# Prepare your datasets (train_dataset, test_dataset) as needed
# For example:
# train_dataset = ...
# test_dataset = ...

# Import your custom trainer
from your_trainer_module import GRPOTrainer

# Initialize the trainer using the tokenizer directly without chat templating
trainer = GRPOTrainer(
    model=model,
    processing_class=tokenizer,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
    # Add other necessary arguments
)

# Start training
trainer.train()

In [ ]:
# --- Modified Input Preparation Function ---
# In your custom GRPOTrainer (likely in UnslothGRPOTrainer.py), locate the _prepare_inputs method.
# Replace the chat template line with the following version that bypasses chat templating:

def _prepare_inputs(self, inputs):
    device = self.accelerator.device
    # Assuming each input is a dict with a 'prompt' key
    prompts = [x["prompt"] for x in inputs]
    
    # Instead of applying a chat template, directly extract the prompt text
    prompts_text = [example["prompt"] for example in inputs]
    
    prompt_inputs = self.processing_class(
        prompts_text, 
        return_tensors="pt", 
        padding=True, 
        padding_side="left", 
        add_special_tokens=False
    )
    
    prompt_inputs = super()._prepare_inputs(prompt_inputs)
    return prompt_inputs

# Make sure your GRPOTrainer uses this updated version.

In [ ]:
# --- Testing Output ---
print("\nTraining Data Test Results:")
for i, (prompt, completion, reward) in enumerate(zip(test_prompts, test_completions, rewards)):
    print(f"\nSample {i+1}:")
    print("Prompt:", prompt[-1]["content"])  # Show last user prompt
    print("Generated:", completion[0]["content"][:100] + "...")
    print("Reference Answer:", test_answers[i][:100] + "...")
    print(f"Reward: {reward:.4f}")